# Entremamiento del modelo

In [5]:
import numpy as np
import pandas as pd
import json
import sys

# Añadir el directorio raíz al path para poder importar módulos personalizados
sys.path.append('..')

from utils import config
from utils import neural_network_functions as nnf
from utils import preprocessing_functions as pf
from utils import evaluation_functions as ef

In [9]:
# 1. Cargar los datos de entrenamiento y validación
# Cargar el dataset normalizado
train_df = pd.read_csv("../data/processed/train_set_normalized.csv")

# Separar características (X) y etiquetas (y)
X = train_df.drop('diagnosis', axis=1).values
y = train_df['diagnosis'].values

# Convertir etiquetas a formato adecuado
y = y.reshape(-1, 1)

# Dividir en conjunto de entrenamiento y validación
X_train, X_valid, y_train, y_valid = pf.train_test_split(X, y, test_size=0.2, random_state=config.RANDOM_STATE)

print(f"x_train shape : {X_train.shape}")
print(f"x_valid shape : {X_valid.shape}")

# 2. Definir la arquitectura de la red neuronal
input_shape = X_train.shape[1]
output_shape = 1

network = nnf.createNetwork([
    nnf.denseLayer(input_shape, 25, activation="sigmoid", weights_initializer="he_uniform"),
    nnf.denseLayer(25, 25, activation="sigmoid", weights_initializer="he_uniform"),
    nnf.denseLayer(25, output_shape, activation="sigmoid", weights_initializer="he_uniform")
])

# 3. Definir hiperparámetros y parámetro de loss function
learning_rate = config.LEARNING_RATE
epochs = config.EPOCHS
batch_size = config.BATCH_SIZE
loss_function = "binary_crossentropy"

# 4. Entrenar el modelo
history = nnf.fit(network, X_train, y_train, X_valid, y_valid,
                    loss=loss_function,
                    learning_rate=learning_rate,
                    batch_size=batch_size,
                    epochs=epochs)

# 5. Guardar el modelo entrenado en formato JSON
model_filepath = "./output/models/saved_model.json" # Cambiamos la extensión a .json
nnf.guardar_modelo_json(network, model_filepath)
print(f"# saving model \"{model_filepath}\" to disk...")

# 6. Calcular Accuracy en el conjunto de validación
predictions_valid = nnf.predict(network, X_valid) # Función predict que debes implementar
accuracy_valid = ef.accuracy(y_valid, predictions_valid) # Usamos la función accuracy de evaluation_functions
print(f"Validation Accuracy: {accuracy_valid:.4f}")


# 7. Visualizar el entrenamiento (opcional)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

AttributeError: module 'utils.config' has no attribute 'RANDOM_STATE'